In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from mySVM.model import SVM
import itertools 
import multiprocessing as mp
import numpy as np

In [2]:
feature_dir = 'Data/FeatureVectors/iFeatureVectors/'
feature_files = [f.name for f in os.scandir(feature_dir)]

In [3]:
label_dir = 'Data/Labels/'
label_file = 'Labels_BinaryClassification.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_dir+label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<40:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=40
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_test,y_train,y_test,verbose=False,optimize=False, pca_comp=n_comp)
    return svm.acc_train,svm.acc_test
 

In [5]:
mp.cpu_count()

24

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(10000)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 3.19 s, sys: 1.52 s, total: 4.7 s
Wall time: 55min 20s


In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('Geary')

{'train_min': 0.7586206896551724,
 'train_mean': 0.8337264367816091,
 'train_max': 0.9540229885057471,
 'train_std': 0.02602079100018547,
 'test_min': 0.4482758620689655,
 'test_mean': 0.7507551724137931,
 'test_max': 0.9655172413793104,
 'test_std': 0.07223516877262756}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('Data/SimResults/iFeatResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
GAAC,0.597701,0.691525,0.793103,0.025670,0.275862,0.640914,0.931034,0.077107
CTDC,0.666667,0.752159,0.919540,0.028427,0.344828,0.681031,0.931034,0.080738
PAAC,0.666667,0.764720,0.942529,0.030294,0.310345,0.678541,0.965517,0.081646
AAC,0.678161,0.778366,0.942529,0.032359,0.344828,0.686776,0.965517,0.080089
APAAC,0.666667,0.779836,0.954023,0.030817,0.310345,0.684776,0.965517,0.082084
CTDD,0.666667,0.789987,0.908046,0.026999,0.310345,0.687793,0.965517,0.080460
QSOrder,0.701149,0.792223,0.977011,0.032072,0.379310,0.697752,0.965517,0.079951
GDPC,0.701149,0.792529,0.908046,0.025494,0.379310,0.665748,0.931034,0.079799
NMBroto,0.747126,0.825201,0.931034,0.022825,0.413793,0.745969,0.965517,0.073906
Geary,0.758621,0.833726,0.954023,0.026021,0.448276,0.750755,0.965517,0.072235
